# Discord

This notebook shows how to use the Discord chat loader. This class helps map from copy-pasted messages (from dms) to a list of langchain messages.

The process has three steps:
1. Create the chat .txt file by copying chats from the Discord app and pasting them in a file on your local computer
2. Create the `DiscordChatLoader` with the file path pointed to the text file.
3. Call `loader.load()` (or `loader.lazy_load()`) to perform the conversion.

## 1. Creat message dump

Currently (2023/08/23) this loader only supports .txt files in the format generated by copying messages in the app to your clipboard and pasting in a file. Below is an example.

In [8]:
%%writefile discord_chats.txt
talkingtower — 08/15/2023 11:10 AM
Love music! Do you like jazz?
reporterbob — 08/15/2023 9:27 PM
Yes! Jazz is fantastic. Ever heard this one?
Website
Listen to classic jazz track...

talkingtower — Yesterday at 5:03 AM
Indeed! Great choice. 🎷
reporterbob — Yesterday at 5:23 AM
Thanks! How about some virtual sightseeing?
Website
Virtual tour of famous landmarks...

talkingtower — Today at 2:38 PM
Sounds fun! Let's explore.
reporterbob — Today at 2:56 PM
Enjoy the tour! See you around.
talkingtower — Today at 3:00 PM
Thank you! Goodbye! 👋
reporterbob — Today at 3:02 PM
Farewell! Happy exploring.

Writing discord_chats.txt


## 2. Create loader

We will point to the file we just wrote to disk.

In [9]:
from langchain.chat_loaders.discord import DiscordChatLoader

In [13]:
loader = DiscordChatLoader(
    path="./discord_chats.txt", 
    user_name="talkingtower", # the user handle to represent the ai message
    merge_runs=True, # The default. This combines adjacent messages into a single one with double-newline separation
)

## 3. Load Messages

Assuming the format is correct, the loader will convert the chats to langchain messages.

In [16]:
from typing import List
from langchain.chat_loaders.base import ChatSession

messages: List[ChatSession] = loader.load()

In [17]:
messages

[{'messages': [AIMessage(content='Love music! Do you like jazz?', additional_kwargs={'sender': 'talkingtower', 'events': [{'message_time': '08/15/2023 11:10 AM\n'}]}, example=False),
   HumanMessage(content='Yes! Jazz is fantastic. Ever heard this one?\nWebsite\nListen to classic jazz track...', additional_kwargs={'sender': 'reporterbob', 'events': [{'message_time': '08/15/2023 9:27 PM\n'}]}, example=False),
   AIMessage(content='Indeed! Great choice. 🎷', additional_kwargs={'sender': 'talkingtower', 'events': [{'message_time': 'Yesterday at 5:03 AM\n'}]}, example=False),
   HumanMessage(content='Thanks! How about some virtual sightseeing?\nWebsite\nVirtual tour of famous landmarks...', additional_kwargs={'sender': 'reporterbob', 'events': [{'message_time': 'Yesterday at 5:23 AM\n'}]}, example=False),
   AIMessage(content="Sounds fun! Let's explore.", additional_kwargs={'sender': 'talkingtower', 'events': [{'message_time': 'Today at 2:38 PM\n'}]}, example=False),
   HumanMessage(content

### Next Steps

You can then use these messages how you see fit, such as finetuning a model, few-shot example selection, or directly make predictions for the next message  

In [22]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()

for chunk in llm.stream(messages[0]['messages']):
    print(chunk.content, end="", flush=True)

Thank you! Have a wonderful day!